In [239]:
import pandas as pd
import json
import requests
import math

In [240]:
#Global paramenters
#Name of drug changes
base_url = "https://rxnav.nlm.nih.gov/REST/"
getDrugs = "drugs.json?name="
df = pd.DataFrame()


In [241]:
df

""


In [242]:
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+getDrugs+name_of_drug)
    rd_df = pd.DataFrame(rd_df.drugGroup.conceptGroup)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
        row += 1
    return df

In [244]:
query_drug('hydrocodone')

KeyboardInterrupt: 

In [ ]:
df = query_drug('hydrocodone')
df.head()

In [ ]:
#loop array to get complete list
df_opioid = pd.DataFrame()
drugs = ['hydrocodone', 'fentanyl']
for drug in drugs:
    merg_df = query_drug(drug)
    df_opioid = pd.concat([df, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_opioid.head()

In [ ]:
#Export
df_opioid.to_excel(r"/Users/hm/Documents/opioid_list/opioid_list/New_Complete_Opioids_List.xlsx", index=False)

In [245]:
df_opioid

,rxcui,name
0,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...
1,1087427,12 HR chlorpheniramine polistirex 8 MG / hydro...
2,1087463,12 HR chlorpheniramine polistirex 1.6 MG/ML / ...
3,1310202,acetaminophen 300 MG / hydrocodone bitartrate ...
4,1310270,acetaminophen 300 MG / hydrocodone bitartrate ...
...,...,...
189,858087,fentanyl 1.2 MG Buccal Film
190,858092,fentanyl 0.2 MG Buccal Film
191,858095,fentanyl 0.4 MG Buccal Film
192,858098,fentanyl 0.6 MG Buccal Film


In [246]:
#take list and format it to the RXNorm api format

#This list was made by cobing the excel files
df_list = pd.read_excel("/Users/hm/Documents/opioid_list//opioid_list/Complete_Opioids_List.xlsx")
df_list = df_list[['Code', 'Description']]
df_list.rename(columns= {'Code' : 'rxcui', 'Description': 'name'}, inplace = True)
df_list

,rxcui,name
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...
...,...,...
5153,996716,codeine phosphate 20 MG / guaifenesin 400 MG /...
5154,996727,codeine phosphate 20 MG / guaifenesin 400 MG O...
5155,996730,codeine phosphate 20 MG / pseudoephedrine hydr...
5156,996738,codeine phosphate 9 MG / guaifenesin 200 MG Or...


In [247]:
#This list was created by pulling from the RxNorm API
df_opioid = df_opioid[['rxcui', 'name']]
df_opioid

,rxcui,name
0,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...
1,1087427,12 HR chlorpheniramine polistirex 8 MG / hydro...
2,1087463,12 HR chlorpheniramine polistirex 1.6 MG/ML / ...
3,1310202,acetaminophen 300 MG / hydrocodone bitartrate ...
4,1310270,acetaminophen 300 MG / hydrocodone bitartrate ...
...,...,...
189,858087,fentanyl 1.2 MG Buccal Film
190,858092,fentanyl 0.2 MG Buccal Film
191,858095,fentanyl 0.4 MG Buccal Film
192,858098,fentanyl 0.6 MG Buccal Film


In [248]:
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame({"drugs_in_list_rxcui": [], "drugs_in_list_name": []})
new_drug = pd.DataFrame({"new_drugs_rxcui": [], "new_drugs_name": []})
for code in df_opioid['rxcui']:
    drug_name = df_opioid['name'].where(df_opioid['rxcui'] == code).dropna().values[0]
    if df_list.isin([int(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = same_drug.append({'drugs_in_list_rxcui': code, 'drugs_in_list_name' : drug_name}, ignore_index = True)
    else:
        new_drug_count += 1
        new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  191
New drug count:  3


In [249]:
#Displays same and new drugs
df_final = same_drug.join(new_drug)
df_final.head()

""


In [250]:
df_final.head()

,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...,992660,homatropine methylbromide 1.5 MG / hydrocodone...
1,1087427,12 HR chlorpheniramine polistirex 8 MG / hydro...,2474267,2 ML fentanyl 0.05 MG/ML Prefilled Syringe
2,1087463,12 HR chlorpheniramine polistirex 1.6 MG/ML / ...,2474269,1 ML fentanyl 0.05 MG/ML Prefilled Syringe
3,1310202,acetaminophen 300 MG / hydrocodone bitartrate ...,NaN,NaN
4,1310270,acetaminophen 300 MG / hydrocodone bitartrate ...,NaN,NaN


In [252]:
type(code)

int

In [253]:
absent_drugs = pd.DataFrame({"drugs_not_found_rxcui": [], "drugs_not_found_name": []})
count = 0
for code in df_list['rxcui']:
    drug_name = df_list['name'].where(df_list['rxcui'] == code).dropna().values[0]
    if df_final.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = absent_drugs.append({'drugs_not_found_rxcui': str(code), 'drugs_not_found_name': drug_name}, ignore_index = True)
print('Count: ', count)
absent_drugs.head()
        
        

Count:  191


,drugs_not_found_rxcui,drugs_not_found_name
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...


In [260]:
df_final = pd.DataFrame()
df_final = absent_drugs.join(same_drug)
df_final = df_final.join(new_drug)
print(df_final.shape) #check if correct size
df_final.head()

(4967, 6)


,drugs_not_found_rxcui,drugs_not_found_name,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...,992660,homatropine methylbromide 1.5 MG / hydrocodone...
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,1087427,12 HR chlorpheniramine polistirex 8 MG / hydro...,2474267,2 ML fentanyl 0.05 MG/ML Prefilled Syringe
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,1087463,12 HR chlorpheniramine polistirex 1.6 MG/ML / ...,2474269,1 ML fentanyl 0.05 MG/ML Prefilled Syringe
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...,1310202,acetaminophen 300 MG / hydrocodone bitartrate ...,NaN,NaN
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...,1310270,acetaminophen 300 MG / hydrocodone bitartrate ...,NaN,NaN


In [261]:
#export
df_final.to_excel(r"/Users/hm/Documents/opioid_list/opioid_list/Opioids_Comparison.xlsx", index=False)

In [271]:
df_final

,drugs_not_found_rxcui,drugs_not_found_name,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...,992660,homatropine methylbromide 1.5 MG / hydrocodone...
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,1087427,12 HR chlorpheniramine polistirex 8 MG / hydro...,2474267,2 ML fentanyl 0.05 MG/ML Prefilled Syringe
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,1087463,12 HR chlorpheniramine polistirex 1.6 MG/ML / ...,2474269,1 ML fentanyl 0.05 MG/ML Prefilled Syringe
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...,1310202,acetaminophen 300 MG / hydrocodone bitartrate ...,NaN,NaN
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...,1310270,acetaminophen 300 MG / hydrocodone bitartrate ...,NaN,NaN
...,...,...,...,...,...,...
4962,996716,codeine phosphate 20 MG / guaifenesin 400 MG /...,NaN,NaN,NaN,NaN
4963,996727,codeine phosphate 20 MG / guaifenesin 400 MG O...,NaN,NaN,NaN,NaN
4964,996730,codeine phosphate 20 MG / pseudoephedrine hydr...,NaN,NaN,NaN,NaN
4965,996738,codeine phosphate 9 MG / guaifenesin 200 MG Or...,NaN,NaN,NaN,NaN


In [310]:
#start here for absolete
#reduce data to be used to find obsolete
not_found = df_final[['drugs_not_found_rxcui', 'drugs_not_found_name']]
not_found.rename(columns= {'drugs_not_found_rxcui' : 'rxcui', 'drugs_not_found_name': 'name'}, inplace = True)
not_found

/Users/hm/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,rxcui,name
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...
...,...,...
4962,996716,codeine phosphate 20 MG / guaifenesin 400 MG /...
4963,996727,codeine phosphate 20 MG / guaifenesin 400 MG O...
4964,996730,codeine phosphate 20 MG / pseudoephedrine hydr...
4965,996738,codeine phosphate 9 MG / guaifenesin 200 MG Or...


In [315]:
df_status = pd.DataFrame()
for code in not_found['rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    if(df_type.shape[0] != 1):
        print('error')
    df_status = df_status.append(df_type)

KeyboardInterrupt: 

In [316]:
df_status

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
0,Active,RXNORM,102010,,YES,102010,,,1010600
0,Active,RXNORM,102010,,YES,102010,,,1010604
0,Active,RXNORM,112010,,YES,112010,,,1014599
0,Active,RXNORM,112010,,YES,112010,,,1014615
0,Active,RXNORM,112010,,YES,112010,,,1014632
...,...,...,...,...,...,...,...,...,...
0,Active,RXNORM,082014,,YES,082014,,,1542997
0,Active,RXNORM,092014,,YES,092014,,,1544851
0,Active,RXNORM,092014,,YES,092014,,,1544854
0,Active,RXNORM,012015,,YES,012015,,,1596108


In [317]:
df_not_found = not_found.merge(df_status, on='rxcui')

In [318]:
df_not_found.head()

,rxcui,name,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,Active,RXNORM,102010,,YES,102010,,
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,Active,RXNORM,102010,,YES,102010,,
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,Active,RXNORM,112010,,YES,112010,,
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...,Active,RXNORM,112010,,YES,112010,,
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...,Active,RXNORM,112010,,YES,112010,,


In [319]:
#export
df_not_found.to_excel(r"/Users/hm/Documents/opioid_list/opioid_list/Status_of_Not_found.xlsx", index=False)